<a href="https://colab.research.google.com/github/LeeSeungwon89/Machine-learning_Theory/blob/master/CHAPTER3%20%ED%8F%89%EA%B0%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **들어가며**

# **1. 정확도(Accuracy)**

# **2. 오차 행렬**

# **3. 정밀도와 재현율**

## **3.1. 정밀도/재현율 트레이드오프**

## **3.2. 정밀도와 재현율의 맹점**

### **3.2.1. 정밀도가 100%가 되는 방법**

### **3.2.2. 재현율이 100%가 되는 방법**

# **4. F1 스코어**

# **5. ROC 곡선과 AUC**

# **6. 파마 인디언 당뇨병 예측**

# **7. 정리**